In [1]:
import random
import math

class Server:
    """Сервер, обрабатывающий задачи.
    Хранит список обработанных задач в виде кортежей (время_начала, время_конца).
    """
    def __init__(self):
        self.processings_list = []  # список кортежей (start, end)

    def try_process(self, t_arrival: float, t_service: float) -> bool:
        """
        Пытается обработать задачу, пришедшую в момент t_arrival.
        Если сервер свободен (нет задач или последняя задача уже завершилась к моменту t_arrival),
        то задача принимается, в список добавляется (t_arrival, t_arrival + t_service),
        и возвращается True. Иначе возвращается False.
        """
        if not self.processings_list or t_arrival >= self.processings_list[-1][1]:
            # Сервер свободен: начинаем обработку немедленно
            self.processings_list.append((t_arrival, t_arrival + t_service))
            return True
        else:
            # Сервер занят → отказ
            return False

    def get_busy_intervals(self):
        """Возвращает список интервалов занятости сервера (копию)."""
        return self.processings_list.copy()


def generate_linear(total_time: float, tz_min: float, tz_max: float, ts_min: float, ts_max: float):
    """
    Генерация для линейного (равномерного) закона распределения.
    Возвращает (arrival_times, service_times) – два списка одинаковой длины.
    """
    arrival_times = []
    service_times = []
    current_time = 0.0
    while current_time < total_time:
        # интервал до следующей заявки
        interval = random.uniform(tz_min, tz_max)
        current_time += interval
        if current_time < total_time:
            arrival_times.append(current_time)
            # время обработки
            service_times.append(random.uniform(ts_min, ts_max))
    return arrival_times, service_times


def generate_exponential(total_time: float, lam: float, mean_service: float):
    """
    Генерация для экспоненциального закона:
    - входной поток с интенсивностью lam (средний интервал 1/lam)
    - время обработки с экспоненциальным распределением, среднее = mean_service
    Возвращает (arrival_times, service_times).
    """
    arrival_times = []
    service_times = []
    current_time = 0.0
    while current_time < total_time:
        # интервал до следующей заявки (экспоненциальный)
        interval = random.expovariate(lam)  # среднее 1/lam
        current_time += interval
        if current_time < total_time:
            arrival_times.append(current_time)
            # время обработки (экспоненциальное со средним mean_service)
            # В random.expovariate параметр lam = 1/mean_service
            service_times.append(random.expovariate(1.0 / mean_service))
    return arrival_times, service_times


def collect_state_probabilities(server1: Server, server2: Server, sim_time: float):
    """
    Вычисляет вероятности P0, P1, P2 методом интервалов между изменениями состояний.
    Возвращает (p0, p1, p2).
    """
    # Собираем все ключевые моменты времени:
    stamps = [0.0, sim_time]
    for s in [server1, server2]:
        for start, end in s.get_busy_intervals():
            stamps.append(start)
            stamps.append(end)
    stamps.sort()

    p0_time = p1_time = p2_time = 0.0

    # Проходим по интервалам (stamps[i], stamps[i+1])
    for i in range(len(stamps) - 1):
        t_start = stamps[i]
        t_end = stamps[i + 1]
        if t_end - t_start < 1e-12:
            continue  # защита от нулевых интервалов

        # Берём среднюю точку интервала – состояние на всём интервале постоянно
        t_mid = (t_start + t_end) / 2.0

        # Проверяем занятость каждого сервера в момент t_mid
        busy1 = any(start <= t_mid < end for start, end in server1.get_busy_intervals())
        busy2 = any(start <= t_mid < end for start, end in server2.get_busy_intervals())

        busy_count = (1 if busy1 else 0) + (1 if busy2 else 0)
        delta = t_end - t_start

        if busy_count == 0:
            p0_time += delta
        elif busy_count == 1:
            p1_time += delta
        else:  # busy_count == 2
            p2_time += delta

    return p0_time / sim_time, p1_time / sim_time, p2_time / sim_time


def simulate(generator_func, sim_time: float, description: str):
    """
    Основная функция симуляции.
    generator_func – функция, которая возвращает (arrival_times, service_times)
    """
    print(f"\n--- {description} ---")
    # Генерируем входные данные
    arrival_times, service_times = generator_func()

    # Создаём серверы
    server1 = Server()
    server2 = Server()

    rejected = 0
    total_requests = len(arrival_times)

    # Обрабатываем заявки по порядку
    for t_arr, t_serv in zip(arrival_times, service_times):
        # Сначала пробуем первый сервер
        if server1.try_process(t_arr, t_serv):
            continue  # обработано первым
        # Иначе пробуем второй
        if server2.try_process(t_arr, t_serv):
            continue  # обработано вторым
        # Оба заняты → отказ
        rejected += 1

    processed = total_requests - rejected

    # Сбор статистики
    if total_requests > 0:
        p_otk = rejected / total_requests
        q = processed / total_requests
    else:
        p_otk = 0.0
        q = 0.0

    s = processed / sim_time  # абсолютная пропускная способность (заявок в секунду)

    # Среднее число занятых серверов (через загрузку)
    total_busy_time1 = sum(end - start for start, end in server1.get_busy_intervals())
    total_busy_time2 = sum(end - start for start, end in server2.get_busy_intervals())
    k = (total_busy_time1 + total_busy_time2) / sim_time

    # Вероятности состояний P0, P1, P2
    p0, p1, p2 = collect_state_probabilities(server1, server2, sim_time)

    # Вывод результатов
    print(f"Всего заявок: {total_requests}, обработано: {processed}, отклонено: {rejected}")
    print(f"P0 = {p0:.6f} (вероятность простоя обоих серверов)")
    print(f"P1 = {p1:.6f} (вероятность занятости одного сервера)")
    print(f"P2 = {p2:.6f} (вероятность занятости двух серверов)")
    print(f"Q (относительная пропускная способность) = {q:.6f}")
    print(f"S (абсолютная пропускная способность) = {s:.6f} заявок/сек")
    print(f"Pотк (вероятность отказа) = {p_otk:.6f}")
    print(f"K (среднее число занятых серверов) = {k:.6f}")

    return {
        "P0": p0, "P1": p1, "P2": p2,
        "Q": q, "S": s, "Pотк": p_otk, "K": k
    }


# Параметры симуляции
SIM_TIME = 3600  # 1 час в секундах

# Случай 1: Линейное распределение (по варианту 20)
def linear_case():
    return generate_linear(
        total_time=SIM_TIME,
        tz_min=1/2,   # 0.5 сек
        tz_max=5/6,   # ~0.8333 сек
        ts_min=1,
        ts_max=5
    )

# Случай 2: Экспоненциальное распределение (по варианту 20)
def exponential_case():
    return generate_exponential(
        total_time=SIM_TIME,
        lam=1.5,           # интенсивность входного потока λ = 1.5
        mean_service=2.0   # среднее время обработки
    )

# Запуск симуляции для двух случаев
random.seed(42)  # для воспроизводимости (можно убрать)
results_linear = simulate(linear_case, SIM_TIME, "Линейный закон (исходный вариант)")

random.seed(42)  # сброс seed для чистоты эксперимента
results_exponential = simulate(exponential_case, SIM_TIME, "Экспоненциальный закон (λ=1.5, tобр=2сек)")


--- Линейный закон (исходный вариант) ---
Всего заявок: 5400, обработано: 2116, отклонено: 3284
P0 = 0.014051 (вероятность простоя обоих серверов)
P1 = 0.213696 (вероятность занятости одного сервера)
P2 = 0.772749 (вероятность занятости двух серверов)
Q (относительная пропускная способность) = 0.391852
S (абсолютная пропускная способность) = 0.587778 заявок/сек
Pотк (вероятность отказа) = 0.608148
K (среднее число занятых серверов) = 1.759195

--- Экспоненциальный закон (λ=1.5, tобр=2сек) ---
Всего заявок: 5404, обработано: 2529, отклонено: 2875
P0 = 0.124958 (вероятность простоя обоих серверов)
P1 = 0.358594 (вероятность занятости одного сервера)
P2 = 0.517404 (вероятность занятости двух серверов)
Q (относительная пропускная способность) = 0.467987
S (абсолютная пропускная способность) = 0.702500 заявок/сек
Pотк (вероятность отказа) = 0.532013
K (среднее число занятых серверов) = 1.393402
